# Tech Challenger - Fase 2


O comando abaixo baixa as bibliotecas nas versões que foram usadas nesse projeto, caso queira executar o notebook é importante criar um virtualenv no seu ambiente.
na documentação oficial do python a seguir tem o passo a passo 
https://docs.python.org/pt-br/3/library/venv.html

In [ ]:
!pip install -r requirements.txt

## Importando as bibliotecas

In [1570]:
import arch
import holidays
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from prophet import Prophet
import scipy.stats as stats
from termcolor import colored
from matplotlib import rcParams
import matplotlib.pyplot as plt
from prettytable import PrettyTable
from sklearn.model_selection import train_test_split
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error

## Tabela de Utilidades das Bibliotecas Importadas



| Biblioteca | Descrição | 
|---|---|
| arch | Modelagem de processos ARCH (Autoregressive Conditional Heteroskedasticity) para séries temporais financeiras. |
| holidays | Identifica feriados em diferentes países. |
| numpy (np) | Biblioteca fundamental para operações matemáticas e manipulação de arrays multidimensionais. |
| pandas (pd) |  Biblioteca para análise e manipulação de dados em DataFrames e Series. |
| seaborn (sns) |  Criação de visualizações estatísticas de alta qualidade com base em Matplotlib. |
| xgboost (xgb) | Implementação altamente performática do algoritmo XGBoost para tarefas de aprendizagem supervisionada (regressão e classificação). |
| pmdarima (pm) | Modelagem de séries temporais com modelos ARIMA (Autoregressive Integrated Moving Average) e SARIMA (Seasonal ARIMA) automatizados. |
| date | Manipulação de datas. |
| prophet | Biblioteca para previsão de séries temporais baseada no algoritmo Prophet do Facebook. | 
| scipy.stats (stats) | Funções estatísticas avançadas como testes de hipótese e distribuições de probabilidade. |
| termcolor | Impressão de texto colorido no terminal. |
| matplotlib (plt) | Biblioteca poderosa para criação de gráficos e visualização de dados. |
| rcParams | Funções para configuração global de parâmetros do Matplotlib. |
| prettytable | Criação de tabelas legíveis e formatadas para impressão no terminal. |
| pmdarima.arima.auto_arima | Função para identificação automática de modelos ARIMA para séries temporais. |
| statsmodels.tsa.arima.model.ARIMA | Classe para implementação manual de modelos ARIMA. |
| statsmodels.tsa.stattools  | Funções para testes de estacionariedade de séries temporais (ADF, KPSS). |
| statsmodels.tsa.seasonal | Função para decomposição sazonal de séries temporais. |
| statsmodels.graphics.tsaplots | Funções para plotagem de funções de autocorrelação (ACF) e autocorrelação parcial (PACF) de séries temporais. |
| sklearn.metrics | Funções para cálculo de métricas de erro em tarefas de previsão (MAE, RMSE, MAPE). |



## Definições Padrões para uso do matplotlib

In [1571]:
rcParams['figure.figsize'] = 12, 5

A configuração acima foi definida para manter a escala de 12 por 5 para todos os plot do projeto

# Importando os dados do IBOVESPA 

In [ ]:
df = pd.read_csv("Ibovespa 2004 a 2024.csv", sep=',')
df

#### Os dados acima foram importados do site investing através do link (https://br.investing.com/indices/bovespa-historical-data) conforme solicitado no tech challenger

In [ ]:
df.head()

> Visualização das 5 primeiras linhas

In [ ]:
df.tail()

#### Visualização das 5 ultimas linhas

In [ ]:
df.info()

#### Visualização das informações da tabela, foi notado que o campo data está como objet em vez de datetime[ns]

In [ ]:
df.describe()

#### Visualização de dados estastistico basico da base

# Tratamento de dados

#### A partir do dataframe DF foi criado uma Serie Fechamento, o campo data foi formatado para datetime e definido como index da serie

In [ ]:
serie_fechamento = pd.Series(data=df['Último'].values, index=pd.to_datetime(df['Data'], format='%d.%m.%Y'))
serie_fechamento = serie_fechamento.sort_index(ascending=True)
serie_fechamento

# Avaliando os dados

#### Criamos uma função reutilizável para exibir séries. Ela permite personalizar o título e as legendas, que podem ser passadas como listas

In [1578]:
def plotarSerie(serie, titulo="Valores da bolsa de valores com base no fechamento diário", legendas=[]):
    serie.plot()    
    plt.title(titulo)
    plt.xlabel('Data')
    plt.grid(visible=True, linestyle='-', linewidth=0.7)
    plt.ylabel('Valor em Real (R$)')
    if legendas:
        plt.legend(labels=legendas, loc='best')

#### Feito um primeiro plot utilizando o metodo *plotarSerie(serie, titulo, legendas)* para a serie fechamento

In [ ]:
plotarSerie(serie_fechamento)

#### Já é possível notar que a serie tem tendencia de crescimento e a principio não aparenta ser uma serie estácionaria, devido sua crescente ao longo do tempo

#### A seguir, foi desenvolvido um novo método que irá gerar gráficos da série temporal, incluindo a média móvel mensal, trimestral e anual.
> Obs: estamos considerando que os dias uteis do mes são 22 dias. portanto os numeros do metódo são baseado em dias úteis


In [1580]:
def plotar_medias_moveis(serie, titulo ='Valores da bolsa de valores com média movel' ):
    dias_uteis = 22
    serie_media_movel_30 = serie.rolling(dias_uteis).mean()
    serie_media_movel_90 = serie.rolling(dias_uteis * 3).mean()
    serie_media_movel_anual = serie.rolling(dias_uteis * 12).mean()
    plotarSerie(serie)
    plotarSerie(serie_media_movel_30)
    plotarSerie(serie_media_movel_90)
    plotarSerie(serie_media_movel_anual, titulo ,['Valor Real','Média movel mensal', 'Média Móvel trimestral', 'Média Móvel Anual'])
    plt.show()

#### Utilizando o método descrito anteriormente, vamos plotar a série de fechamento para analisar visualmente sua tendência ao longo do tempo. Para isso, adicionaremos uma linha representando a média móvel de X períodos, o que nos permitirá identificar padrões e possíveis anomalias nos dados

In [ ]:
plotar_medias_moveis(serie_fechamento)

### Agora que já visualizamos os dados, vamos entender como está a distribuição dos dados

In [ ]:
stats.probplot(serie_fechamento, dist='norm', plot=plt)
plt.show()

\- Avaliando a distribuição de dados Q-Q plot (quantile-quantile plot), é possível notar que a distribução dos dados não está normalizada, o esperado é que os pontos azuis, estivessem muitos proximos da linha vermelha. Neste gráfico, os dados começam a se alinhar com a linha vermelha nos quantis intermediários, mas desviam-se nas caudas (extremos), indicando que a distribuição dos dados pode não ser perfeitamente normal.

In [ ]:
sns.histplot(serie_fechamento, kde=True)
plt.show()

\- Uma segunda forma de avaliação é utilizando o histograma, nele é possível visualizar que existem picosem torno dos 20 60 e 110, aparenta ter grupos distintos de dados ou várias distribuições sobre postas, confirmando que os dados não seguem uma distribuição normal. um grafico de distribuição normal, tem uma aparencia em formato de um sino.

## Test de normalidade

\- Muitas vezes as observações de um grafico aparenta ter uma distribuição normais, mas isso não garante que os dados tenho uma distribuição normal, para garantir a distribuição podemos fazer o teste de **Shapiro Wilk**.

## Teste Shapiro-Wilk

### O que é o teste de Shapiro-Wilk?

    O teste de Shapiro-Wilk é uma ferramenta estatística utilizada para avaliar se um conjunto de dados se ajusta a uma distribuição normal. A distribuição normal, também conhecida como curva em forma de sino, é uma das distribuições mais comuns em estatística e é frequentemente utilizada em diversos modelos estatísticos.

#### Por que é importante testar a normalidade?

    Muitos testes estatísticos assumem que os dados seguem uma distribuição normal. Se essa premissa não for verdadeira, os resultados do teste podem ser enganosos. Ao realizar o teste de Shapiro-Wilk, você verifica se essa suposição é razoável para seus dados.

Critérios:

Nível de significância de 0,05 ou 5% (mais utilizado)

Valor de p < 0.05: Os dados provavelmente não são normalmente distribuídos.
Valor de p ≥ 0.05: Não há evidência suficiente para rejeitar a hipótese de normalidade.

### Abaixo criamos um método que calcula a partir de uma serié se a distribuição é normal ou não

In [1584]:
def validacao_distribuicao(serie):
    print(colored(':: Validação Distribuição (Shapiro-Wilk) ::', 'light_blue', attrs=["bold"]))
    e, p = stats.shapiro(serie)
    print(f'Estátistica do teste: {e}')
    print(f'p-valor: {p}')
    if p > e:
        print(colored('>> Distribuição Normal <<','green'))
    else:
        print(colored('Não há evidência suficiente para rejeitar a hipótese de normalidade','red'))


In [ ]:
validacao_distribuicao(serie_fechamento)

\- Foi executado o teste de **Shapiro Wilk** e foi constatado que não há uma distribuição normal, mas podemos tentar buscar uma normalidade através de transformação logaritmica ou transformação de raiz quadrada ou cubica


# Técnicas de Transformação para Normalização


Para transformar os dados visando uma normalização (ou aproximação de uma distribuição normal), você pode tentar diferentes técnicas de transformação. Aqui estão algumas abordagens comuns:


### 1. Transformação Logarítmica
Utilizada para reduzir a assimetria positiva (cauda longa à direita).

\[
    $y' = \log(y)$
\]

**Uso**: Quando os dados possuem muitos valores altos que puxam a média.

---

### 2. Transformação de Raiz Quadrada ou Cúbica
Suaviza variações em dados positivos e assimétricos.

\[
$y' = \sqrt{y}$
\]


## Aplicando a transformação com Logarítmica

In [ ]:
serie_log = np.log(serie_fechamento)
serie_log.head()

In [ ]:
stats.probplot(serie_log, dist='norm', plot=plt)
plt.plot()

In [ ]:
sns.histplot(serie_log, kde=True)

- Após a transformações a distribuição foi modificada, porem não teve bons resultados, pois ela ainda não teve sua distribuição normalizada, iremos aplica a validação de shapiro para confirmação

In [ ]:
validacao_distribuicao(serie_log)

    - Mesmo efetuando transformação em logaritmo não foi possível obter um bom resultado

## Aplicando Transformação de Raiz Quadrada ou Cúbica

Iremos elevar ao cubo para tentar obter uma melhor distribuição, usaremos o sign e abs para evitar o valores negativos

In [ ]:
serie_ao_cubo = np.sign(serie_fechamento)*abs(serie_fechamento)**(1/3)
serie_ao_cubo.head()

In [ ]:
stats.probplot(serie_ao_cubo, dist='norm', plot=plt)
plt.plot()

In [ ]:
sns.histplot(serie_ao_cubo, kde=True)
plt.plot()

In [ ]:
validacao_distribuicao(serie_ao_cubo)

- Mesmo fazendo a transformação para Não foi possível obter a distribuição normal usando serie ao cubo

# Decomposição

#### Decompondo a serie fechamento

- Vamos começar a avaliar os dados separadamente, com objetivo de isolar os componentes para uma analise mais individual o que irá ajuda a identificar o que está impulsionando o comportamento da série, como padrões sazonais ou uma tendência geral.

In [ ]:
serie_decomposta = seasonal_decompose(serie_fechamento, period=22)
serie_decomposta.plot()
plt.tight_layout()

O grafico a acima demonstra uma tendencia crescente ao longo do tempo, mas a sanonalidade fica dificil de avaliar, portanto vamos aumentar o periodo para melhorar a visualização da sazonalidade 

In [ ]:
serie_decomposta = seasonal_decompose(serie_fechamento, model="multiplicative", period=252)
serie_decomposta.plot()
plt.tight_layout()

- Após aumentar o periodo foi possível identificar melhor que existe uma sazonalidade entre os dados e os residuos tambem ficou mais claro
- Os resíduos representam o ruído da série temporal, ou seja, as variações que não são previsíveis com base nos componentes sistemáticos (tendência e sazonalidade, por exemplo). Essas flutuações podem ser devidas a eventos externos, fatores aleatórios ou variações de curto prazo.
- Tantos nos dados observados, tendencia e nos residos é possível notar duas quedas, entre os anos de 2008-2009 e 2020.
- Essas quedas foram acometidas por eventos que impactou a economia mundial, sendo o primeiro deles em 2008 que ocorreu a **crise financeira global** esse evento teve um impacto profundo na economia global e nos mercados financeiros, levando a uma das maiores quedas nas bolsas de valores em todo o mundo desde a Grande Depressão de 1929.
- E em 2020 foi a *pandemia* do da COVID-19. A disseminação rápida do novo coronavírus **(SARS-CoV-2)** causou uma crise global de saúde pública, levando a grandes incertezas econômicas e a um colapso nos mercados financeiros. A crise desencadeada pela COVID-19 afetou praticamente todas as economias e setores ao redor do mundo.

- Para uma observação mais proximas dos dias atuais, iremos observar a sazonalidade nos seus ultimos 500 dias equivalente a 2 anos

In [ ]:
serie_decomposta.seasonal.iloc[-500:].plot()
plt.show()

em 2 anos é possível notar que existe uma sazonalidade entre esse periodo observado

# Estacionaridade

A **estacionaridade** é um conceito fundamental em séries temporais e análises estatísticas, pois tem implicações significativas para a modelagem e previsão de dados.

- Uma série temporal é dita estacionária se suas propriedades estatísticas, como média, variância e autocovariância, permanecem constantes ao longo do tempo. Isso significa que as flutuações e padrões observados em uma parte da série serão semelhantes em outras partes.
- Muitos modelos estatistico  pressupoe que os dados são estacionários, tendo assim uma previsão mais precisa sobre os dados
- A **estacionaridade** permite detectar padrões tendencias e sazonalidades que são essenciais para analise e previsão

Através da decomposição é possível notar que existe uma sazonalidade e uma tendencia de crescimento, que aparenta não ser estácionada.
Para validar iremos um o test de KPSS e Dict Fuller

## Teste KPSS (Kwiatkowski-Phillips-Schmidt-Shin)

- O teste KPSS (Kwiatkowski-Phillips-Schmidt-Shin) é um método estatístico para verificar a estacionaridade de séries temporais. Sua hipótese nula assume que a série é estacionária, enquanto a alternativa sugere que é não estacionária. O teste calcula um estatístico KPSS com base nos resíduos da série e compara com valores críticos para tomar decisões. Se o valor calculado for maior que o crítico, rejeita-se a estacionaridade. É amplamente utilizado em análises econômicas e financeiras para validar a adequação de modelos que presumem estacionaridade.

- Assumos então os seguintes critérios: 

> Ho = não é estacionário: estatística do teste > valor crítico

> Ha = é estacionário:  estatística do teste < valor crítico

#### Validação KPSS

    Para facilitar o reuso criamos um metodo que valida se a serie é ou não estacionária e exibe os valores estatisticos e criticos do teste

In [1597]:
def validacao_kpss(serie):
    print(colored(':: Validação KPSS (Kwiatkowski-Phillips-Schmidt-Shin) ::', 'light_blue', attrs=["bold"]))
    resultado = kpss(serie)
    estatistica_teste = resultado[0]
    p_valor = resultado[1]
    valores_criticos  = resultado[3]
    percentil_referencia = '5%'

    print(f'Estatística do teste: {estatistica_teste:.4f}')
    print(f'p-valor {p_valor:.4f}')
    print('Valores Críticos:')
    table = PrettyTable(['Criticidade', 'Valor'])

    for chave, valor in valores_criticos.items():
        table.add_row([chave, valor])
    
    print(table)

    if estatistica_teste > valores_criticos[percentil_referencia]:
        print(colored('>> Serie não estacionária <<', 'red'))
    else:
        print(colored('>> Série estácionaria <<', 'green'))


#### Aplicando a validação KPSS

In [ ]:
validacao_kpss(serie_fechamento)

- Na aplicação da serie fechamento foi identificada que ela não é estacionária, pois o valor este estátistico é maior que o valor critico

### Teste ADF (Dickey-Fuller)

* O teste de Dickey-Fuller é uma ferramenta estatística usada para verificar se uma série temporal possui raiz unitária, ou seja, se ela é estacionária. 
* A estacionariedade é importante para a aplicação de muitos modelos econométricos. O teste compara uma estatística calculada com um valor crítico. Se a estatística for menor que o valor crítico, rejeitamos a hipótese de raiz unitária e concluímos que a série é estacionária. 
* O teste de Dickey-Fuller aumentado (ADF) é a versão mais comum e inclui termos de defasagem para capturar a autocorrelação. A não estacionariedade pode levar a resultados enganosos em modelos econométricos e comprometer a qualidade das previsões.

> H0: Se a estatística de teste for menor que o valor crítico, rejeitamos a hipótese nula e concluímos que a série é estacionária.
> H0: Se a estatística de teste for maior ou igual ao valor crítico, não rejeitamos a hipótese nula e concluímos que a série pode ter raiz unitária (ou seja, não há evidências suficientes para afirmar que ela é estacionária).

Usaremos como valor de referencia 5%

Resumo: 
- p-valor > 0.05  
    - Se p-valor for maior que o valor de referencia: Não rejeitar a Hipótese Nula: a série não é estacionária 
- p-valor < 0.05
    - se p-valor for menor que o valor de referencia: Rejeitar a Hipótese Nula: a série é estacionária

In [1599]:
def validacao_adfuller(serie):
  print(colored(':: Validação ADF (Dickey-Fuller) ::', 'light_blue', attrs=["bold"]))
  result_adf = adfuller(serie)
  statistic_value = result_adf[0]
  p_value = result_adf[1]
  critical_values =  result_adf[4]
  valor_referencia = 0.05

  print(f"ADF Statistic: {statistic_value}")
  print(f'Valor-p do Teste ADF: {p_value}')

  table = PrettyTable(['Criticidade','Valor'])
  for key, value in critical_values.items():
    table.add_row([key, value])

  print(table)

  if p_value > valor_referencia:
    print(colored('Não rejeitar a Hipótese Nula: a série não é estacionária\n','red'))
  else:
    print(colored('Rejeitar a Hipótese Nula: a série é estacionária\n','green'))

In [ ]:
validacao_adfuller(serie_fechamento)

Usamos o teste de ADF para validar se a serie é ou não estacionária e foi respondido que a serie nao é estácionaria

#### Avaliando a autocorrelação

In [ ]:
plot_acf(serie_fechamento)
plt.show()

- As barras verticais mostram os valores de autocorrelação muito próximos de 1 para todos os lags, exibindo um sinal claro que a série não é estacionária, isso demonstram um padrão de decaimento lento ou até mesmo ausencia de decaimento, como a autocorrelação mantem um valor elevado demonstra que existe uma forte dependencia temporal entre os pontos da série

In [ ]:
plot_pacf(serie_fechamento)
plt.show()

- A autocorrelação parcial apresenta que o valor atual da série está formente relacionado ao valor anterior, com essa informação é possível notar que os valores mais distantes no passado tem pouca influencia na previsão do valor atual

Após todas as analises concluimos que a serie não é estácionaria e que nessecitamos transforma-las em estacionarias para conseguir ter um melhor aproveitamento dos modelos que iremos criar, então iremos aplicar a  técnica de diferenciação

# Diferenciação

- A técnica de diferenciação é uma técnica utilizada para transformar uma série temporal não estacionária em uma série estacionária.
- Ela consiste em subtrair o valor de uma observação do valor da observação anterior. Essa operação remove tendências e sazonalidades presentes na série original, tornando-a mais adequada para a aplicação de modelos econométricos e de previsão. 
- Ao diferenciar uma série, estamos analisando a taxa de variação ao invés dos níveis absolutos dos dados.

#### Aplicando Diferenciação

In [ ]:
serie_fechamento_diff = serie_fechamento.diff()
serie_fechamento_diff = serie_fechamento_diff.dropna()
serie_fechamento_diff

- Foi aplicado a diferenciação e excluido a primeira linha que estava nula, justamente porque na diferenciação se perde o primeiro valor, pois ele não teve nenhum valor para comparar com o anterior tornando-se nulo

In [ ]:
serie_fechamento_diff.plot()
plt.show()

- Após a transformação percebemos que houve uma mudança no comportamento da série, é possível notar a ausencia de tendencia, isso significa que a média e a variancia da série são constantes ao longo do tempo, possívelmente ela se transformou em uma série estácionada, mas para isso iremos aplicar novamente os testes KPSS e o ADF para validar se realmente ela se tornou estácionaria.

In [ ]:
validacao_kpss(serie_fechamento_diff)

- O Resultado da transformação foi positivo, o teste **KPSS** informa que conseguimos tranformar ela em estácionaria

In [ ]:
validacao_adfuller(serie_fechamento_diff)

- O Resultado da transformação foi positivo, o teste **ADF** informa que conseguimos tranformar ela em estácionaria

- Conseguimos através da diferenciação transformar a serie em estácionaria, agora podemos avaliar o comportamento da serie novamente

In [ ]:
sns.histplot(serie_fechamento_diff, kde=True)
plt.plot()

- Avaliando o histograma acima é possivel notar a diferença da serie, mostrando que os dados estão mais centralizado proximo a zero, com formato de sino, indicando uma normalidade entre os dados não perfeita

# Decomposição da serie Diferencial

In [ ]:
serie_diff_decomposta = seasonal_decompose(serie_fechamento_diff, period=365)
serie_diff_decomposta.plot()
plt.tight_layout()

- Após a tranformação tambem é possível notar que não há mais a tendencia de alta e baixa e que os residuos estão estão aleatorios e sem padrões claros, porem é possível notar que os anos de 2008-2009 e 2020 continuam influenciar os graficos de observação, tendencia e residuos

##  Avalição residuos

In [ ]:
sns.histplot(serie_diff_decomposta.resid, kde=True)
plt.show()

- Conseguimos avaliar visualmente que o residuo aparenta ter uma distribuição bem proxima do normal com uma concentração de valores bem proximas a zero, porem a cauda indica possíveis outliers

In [ ]:
plotar_medias_moveis(serie_fechamento_diff, 'Valores da bolsa de valores com média movel com diferenciação')

- Na imagem acima conseguimos visualizar o valor real diferenciado e seus médias moveis mensal, trimestral e anual, onde ela demonstra se tornado estacionária, não tendo evidencias de tendencias, ausencia de padrões sazonais nas médias móveis que sugere tambem que a sazonalidade tambem foi removida e que a variabilidade da série parece ser constante ao longo do tempo

In [ ]:
plotar_medias_moveis(serie_fechamento_diff.loc[pd.Timestamp('2020-01-02'):pd.Timestamp('2023-12-29')], 'Valores da bolsa de valores com média movel com diferenciação de 2020 a 2023')

- Na imagem acima tem a mesmas caracteristica da imagem anterior, a diferença é que estamos vendo os ultimos 4 anos o que torna mais facil a visualização e entendimento do gráfico

# Autocorrelação com diferenciação


In [ ]:
plot_acf(serie_fechamento_diff)
plt.show()

- A autocorrelação da diferenciação apresentou um resultado satisfatórios, pois foi possivel notar que a partir do indicador 2 a correlação se manteve proximo ao intervalo de confiança, exceto por um item fora, porem aceitavel

In [ ]:
plot_pacf(serie_fechamento_diff)
plt.show()

A correlação parcial tambem teve seus bon resultados considerendo-se que tambem a partir do 2 ponto se manteve dentro ou bem proximo do intervalo de confiança. Com esse resultado já é possível utilizar um modelo ARIMA com ordem 2 baseado nesse resultado

# Aplicação do Modelo ARIMA

### AutoRegressive Integrated Moving Average (ARIMA)
*É um modelo estatístico amplamente utilizado para analisar e prever séries temporais. As letras que compõem essa sigla possuem significados específicos:*
* AR (AutoRegressiva): Indica que o valor da série em um determinado momento é uma função linear de seus valores anteriores. Ou seja, os valores passados influenciam os valores futuros.
* I (Integrada): Refere-se ao processo de diferenciação aplicado à série temporal. A diferenciação é uma técnica utilizada para tornar a série estacionária, removendo tendências ou sazonalidades.
* MA (Média Móvel): Indica que o valor da série em um determinado momento é uma função linear dos erros aleatórios (ruídos) ocorridos em momentos anteriores



*Um modelo ARIMA é representado por três números: ARIMA(p,d,q).*
* p: Ordem do processo autoregressivo. Indica o número de períodos anteriores que são usados para prever o valor atual.
* d: Grau de diferenciação. Indica o número de vezes que a série é diferenciada para torná-la estacionária.
* q: Ordem do processo de médias móveis. Indica o número de termos de erro anteriores que são incluídos no modelo.

- Considerando os valores encontrados nos resultados anteriores, iremos começar testando o modelo com os seguintes valores p = 2, d = 1, e q = 1

- sendo *p = 2*  que o valor atual vai depender dos 2 valores anteriores, *d = 1* porque usamos a diferenciação uma unica vez e  *q = 1*  que o erro atual depende do erro anterior

## Aplicando o ARIMA

In [ ]:
p,d,q =  2,1,1
modelo_arima = SARIMAX(serie_fechamento_diff, order = (p,d,q), trend='c', enforce_stationarity=True)
resultado = modelo_arima.fit()
print(resultado.summary())

- O IAC encontrado foi de 15174, porem tem alguns warning no qual podemos avaliar 
1. Ele indica que a index de data, não tem uma frequencia, isso ocorre porque na nossa serié temos alguns dias faltando, como feriados e finais de semana, o que impede de ter todos os dias do ano e não conseguir colocar uma frequencia diaria

#### Prevendo 10 dias

In [ ]:
data_inicial = '2024-03-12'
index = pd.date_range(start=data_inicial, periods=10)
forecast_arima = resultado.forecast(10)
forecast_arima
resultado_previsao_arima = pd.Series(data=forecast_arima.values, index=index)

- usamos o metodo forecast para prever os proximos 10 dias

In [ ]:
resultado_previsao_arima.plot()
plt.plot()

- A previsão conseguiu prever os 2 primeiros valores, e depois disso entro em uma linha reta, isso indica que o nosso modelo não está conseguindo prever muito bem com os nossos dados. vamos aplicar o resultado como Garch para tentar melhorar o resultado

#### Aplicando o GARCH

- (Heterocedasticidade Condicional AutoRegressiva Generalizada) é um modelo estatístico utilizado para modelar a volatilidade de séries temporais financeiras. Ele assume que a variância dos retornos de um ativo não é constante ao longo do tempo, mas sim dependente de suas próprias realizações passadas. Em outras palavras, o GARCH captura a ideia de que períodos de alta volatilidade tendem a ser seguidos por outros períodos de alta volatilidade, e o mesmo ocorre com períodos de baixa volatilidade.

iremos usar o residuo do resultado do modelo, o mean = Zero que estamos assumindo qu e a média dos residuos é zero, o vol como 'GARCH', porque queremos usar um modelo GARCH para modelar a volatividade dos residuos que é capaz de capturar a variancia dos erros ao longo do tempo, e os mesmos p e q usado no ARIMA

In [ ]:
resid = resultado.resid
model_garch = arch.arch_model(resid, mean='Zero', vol='GARCH', p=p, q=q).fit()

Após sua finalização iremos tentar prever novamente, só que agora com 30 dias

In [ ]:
data_inicio, data_fim = '2024-03-12','2024-04-10'
forecast_arima = resultado.forecast(steps=30)
forecast_garch = model_garch.forecast(horizon=30)
forecast = forecast_arima + np.sqrt(forecast_garch.variance.values[-1, :]) * np.random.normal(size=30)
serie_prevista_garch = pd.Series(data=forecast.values, index=pd.date_range(data_inicio, data_fim))


- Fizemos novamente o forecast tando do arima tanto do garch e juntamos os valores num só forcast e criamos uma serie prevista com garch

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(serie_fechamento_diff, label='REAL')
plt.plot(serie_prevista_garch, label='Previsão ARIMA + GARCH')
plt.legend()
plt.show()

- Fizemos o plot dos valores reais + previsto e o resultado foi satisfatorio, veremos mais a seguir um pouco mais de perto como ficou a previsão

In [ ]:
plt.plot(serie_fechamento_diff.loc[pd.Timestamp('2023-01-02'):pd.Timestamp('2024-03-12')], label='REAL')
plt.plot(serie_prevista_garch, label='Previsão ARIMA + GARCH')
plt.legend()
plt.show()

Na imagem acima deu para notar a que a previsão conseguiu manter a previsão adequada e seguindo conforme o esperado

### Reversão diferenciação da previsão

In [1621]:
def reverter_diferenciacao(serie,serie_diferenciada):

    ultimo_valor = serie.iloc[-1]
    serie_normalizada = ultimo_valor + serie_diferenciada.cumsum()
    serie_normalizada

    return serie_normalizada    

In [1622]:
previsao_normal = reverter_diferenciacao(serie_fechamento, serie_prevista_garch)

In [ ]:
plotarSerie(serie_fechamento.loc[pd.Timestamp('2023-01-02'):pd.Timestamp('2024-03-12')])
plotarSerie(previsao_normal, titulo='Dados reais + previsão arima + Garch', legendas=['Série Real', 'Previsão arima + Garch'])

### Avaliação da desempenho do modelo

In [ ]:
dias_avaliacao = 30
treino_arima = serie_fechamento_diff.head(len(serie_fechamento_diff) - dias_avaliacao)
treino_arima.head()

In [ ]:
test_arima = serie_fechamento_diff.tail(dias_avaliacao)
test_arima.head()

In [ ]:
p,d,q =  2,1,1

modelo_arima = SARIMAX(treino_arima, order = (p,d,q), trend='c', enforce_stationarity=True)

resultado = modelo_arima.fit(maxiter=100)

print(resultado.summary())

forecast_arima = resultado.forecast(dias_avaliacao)

resid = resultado.resid
model_garch = arch.arch_model(resid, mean='Zero', vol='GARCH', p=p, q=q).fit()
forecast_garch = model_garch.forecast(horizon=dias_avaliacao)

forecast_garch
forecast = forecast_arima + np.sqrt(forecast_garch.variance.values[-1, :]) * np.random.normal(size=dias_avaliacao)
serie_prevista_garch = pd.Series(data=forecast.values, index=test_arima.index)

### Calculo de metricas (Erro Médio Absoluto, Erro Quadrático Médio e Erro Percentual Médio Absoluto)

* **MAE (Erro Médio Absoluto):**  Mede a média dos erros absolutos entre os valores previstos e os reais. É uma métrica robusta que não é influenciada por outliers e fornece uma ideia geral da precisão do modelo.
* **MSE (Erro Quadrático Médio):** Calcula a média dos erros ao quadrado. Penaliza mais os erros maiores, sendo útil quando grandes erros são mais problemáticos. É a base para o RMSE (raiz do erro quadrático médio), que é mais interpretado na mesma escala que os dados originais.
* **MAPE (Erro Percentual Médio Absoluto):** Calcula a média dos erros percentuais absolutos. É útil para comparar a precisão de modelos em diferentes escalas, mas pode ser instável quando há valores próximos de zero.

**Em resumo:** O MAE fornece uma medida de erro geral, o MSE penaliza mais erros grandes e o MAPE expressa o erro como uma porcentagem. A escolha da métrica depende do problema específico e da importância relativa de diferentes tipos de erros.

Para facilitar o Reuso criamos 2 metodos. um que calcula as métricas e outro que imprime o valor das metricas com seus respectivos valores


In [1627]:
def calculate_metrics(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = root_mean_squared_error(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    return mae, mse, mape


def print_metrics(metrics):
    mae, mse, mape = metrics
    print(colored(f"MAE: {mae}", 'light_blue'))
    print(colored(f"MSE: {mse}", 'light_cyan'))
    print(colored(f"MAPE: {mape:.2f} %", 'light_green'))

- Foi criado uma função para salvar em memória os resultados dos testes, para no final compararmos o modelo que obteve o melhor resultado nas previsões

In [1628]:
metricas = {}

def salvar_metricas(modelo, new_metric):
    mae, mse, mape = new_metric
    m = {
        "mae": mae,
        "mse": mse,
        "mape": mape
    }
    
    metricas[modelo]=m


Para obter uma melhor metricas, iremos desfazer a diferenciação, colocando os dados na sua escala real, assim poderemos tirar melhor aproveitamento das metricas

In [1629]:
test_normalizada = reverter_diferenciacao(serie_fechamento.head(len(serie_fechamento) - dias_avaliacao),test_arima )
previsao_normalizada = reverter_diferenciacao(serie_fechamento.head(len(serie_fechamento) - dias_avaliacao),serie_prevista_garch )


In [ ]:
metrics = calculate_metrics(test_normalizada.values, previsao_normalizada.values)
print_metrics(metrics)
salvar_metricas('Arima', metrics)

# Aplicação do modelo XGBRegressor

O **XGBRegressor** é um algoritmo de *machine learning* poderoso e eficiente, especialmente utilizado para **problemas de regressão**. Ele funciona combinando múltiplas árvores de decisão (ensemble) de forma sequencial, aprendendo dos erros dos modelos anteriores. Essa abordagem, chamada de *gradient boosting*, permite que o XGBRegressor construa modelos altamente precisos e robustos. 

Embora o XGBRegressor não tenha sido especificamente projetado para séries temporais, ele pode ser adaptado para esse tipo de problema com excelentes resultados. Isso se deve a sua capacidade de:

**Capturar padrões complexos:** As árvores de decisão do XGBoost são capazes de capturar padrões não-lineares e interações entre diferentes características, o que é comum em séries temporais.

**Lidar com grandes volumes de dados:** O algoritmo é otimizado para lidar com grandes conjuntos de dados, o que é frequentemente o caso em séries temporais.

**Alta precisão:** Em diversas competições de machine learning, o XGBoost tem demonstrado ser um dos algoritmos mais precisos.



#### Preparação dos dados

- No método abaixo estamos criando uma função para preparar os dados para o nosso modelo, separamos a data por ano, mes,dia e dia da semana, e renomeamos duas colunas, Último para valor_fechamento e Abertura para valor_abertura

In [1936]:
def create_feature(dado):
    data_frame = pd.DataFrame(dado)
    data_frame['valor_fechamento']  = data_frame['Último']
    data_frame['valor_abertura']  = data_frame['Abertura']
    data_frame['Data'] = pd.to_datetime(data_frame['Data'], format='%d.%m.%Y')
    data_frame['year'] = data_frame["Data"].dt.year
    data_frame['month'] = data_frame["Data"].dt.month
    data_frame['day'] = data_frame["Data"].dt.day
    data_frame['dayofweek'] = data_frame["Data"].dt.dayofweek
    return data_frame

Com a função criada, criamos duas novas variaveis, que é o feature_names no qual será as variaveis que o modelo irá usar do dataframe e o target, que é o valor que o modelo tentará prever.

In [1937]:
feature_names = ["year", "month", "day", "dayofweek","valor_abertura"]
target = "valor_fechamento"

- Em seguida, criaremos um DataFrame de features e o ordenaremos por data para preparar os dados para a divisão em conjuntos de treino e teste.

In [ ]:
df_feature = create_feature(df)
df_feature = df_feature.sort_values(by='Data')
df_feature

- Com o DataFrame de features pronto, podemos dividir os dados em conjuntos de treino e teste. Utilizaremos a função **train_test_split** do scikit-learn para essa tarefa. A variável **X** conterá as features (características) que o modelo utilizará para fazer as previsões, enquanto **y** conterá o target (valor que queremos prever). Definiremos **test_size=0.0059** para que o conjunto de teste represente os últimos 30 dias dos nossos dados (considerando um DataFrame com 5000 registros). Ao configurar **shuffle=False**, garantimos que a ordem temporal dos dados seja preservada, evitando que os dados de teste 'contaminem' o conjunto de treino.

In [ ]:
X = df_feature[feature_names]
y = df_feature['valor_fechamento']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.0059, shuffle=False)
len(X_train), len(X_test)

Separamos treino e teste e temos 4970 registros para teste e 30 para treino

In [ ]:
X_train.head()

Observando as 5 primeiros linhas do treino deu para validar que os dados começaram de 2004 conforme o  esperavamos

In [ ]:
X_train.tail()

Observando suas 5 últimas linhas do treino foi possível notar que os ultimos dias foram janeiro de 2024 conforme o esperado

In [ ]:
X_test

o nosso X_test tambem ficou adequado, começando final de janeiro até dia 12 de março

Criamos um modelo XGBRegressor configurado para minimizar o erro quadrático médio ("reg:squarederror").
- Treinamos o modelo com os dados de treino (`X_train`, `y_train`). O modelo aprende a relação entre as features (`X_train`) e o target (`y_train`).
- Utilizamos o modelo treinado para fazer previsões nos dados de teste (`X_test`). A variável `predict_values` armazena as previsões do modelo para cada ponto no conjunto de teste.
- `predict_values` contém as previsões do modelo para o target nos dados de teste, que podemos comparar com o target real para avaliar a performance do modelo.


In [1943]:
reg = xgb.XGBRegressor(objective="reg:squarederror")
reg.fit(X_train, y_train)

predict_values = reg.predict(X_test)

Com os dados já previstos, iremos calcular as metricas para entender como foi a performance do modelo

In [ ]:
metrics = calculate_metrics(y_true=predict_values, y_pred=y_test.values)
print_metrics(metrics)
salvar_metricas('XGBRegressor', metrics)

O erro médio absoluto (MAE) indica que, em média, as previsões estão desviando cerca de 1.68 unidades do valor real. 

O erro quadrático médio (MSE) sugere que os erros maiores estão sendo penalizados.

O erro percentual absoluto médio (MAPE) indica um erro médio de aproximadamente 1.32%.

Agora iremos reconstruir a data da serie treino para que seja possível visualizar os dados com os valores previstos

In [ ]:
X_train["data"] = X_train["year"].map(str) + "-" + X_train["month"].map(str) + "-" + X_train["day"].map(str)
serie_treino = pd.Series(data=y_train.values, index=pd.to_datetime(X_train['data']))
serie_treino

Construindo as datas dos valores previsto

In [ ]:
X_test["data"] = X_test["year"].map(str) + "-" + X_test["month"].map(str) + "-" + X_test["day"].map(str)
serie_prevista_xgb = pd.Series(data=predict_values, index=pd.to_datetime(X_test['data']))
serie_prevista_xgb


In [ ]:
plotarSerie(serie_treino)
plotarSerie(serie_prevista_xgb,titulo="Previsão bolsa de valores usando XGB Regressor", legendas=['Valor Real', 'Valor Previsto'])


In [ ]:
plotarSerie(serie_treino[-500:])
plotarSerie(serie_prevista_xgb, titulo="Previsão bolsa de valores usando XGB Regressor", legendas=['Valor Real', 'Valor Previsto'])


# Aplicação do modelo Prophet

O **Prophet** é uma biblioteca desenvolvida pelo Facebook (agora Meta) voltada para a modelagem e previsão de **séries temporais**. Ele foi projetado para lidar com componentes sazonais, tendências e feriados de maneira intuitiva e eficiente. Uma de suas maiores vantagens é a simplicidade e a facilidade de uso, mesmo para aqueles que não possuem experiência avançada com modelagem estatística.

Prophet se destaca por:

- Ser robusto com dados ausentes.
- Gerenciar mudanças de tendência.
- Considerar sazonalidades e eventos especiais, como feriados.
- Funcionar bem com séries temporais diárias, semanais e anuais.

In [ ]:
df.info()

Preparação dos dados para utilização com o modelo, mantendo somente as datas (ds) e o target y (Último):

In [ ]:
df_p = pd.DataFrame(df)
df_p['ds'] = pd.to_datetime(df['Data'], format='%d.%m.%Y')
df_p.head()


In [ ]:
df_p.rename(columns={'Último':'y'}, inplace=True)
df_p

In [1995]:
df_p = df_p[['ds', 'y']]

In [ ]:
df_p = df_p.sort_values(by='ds')
df_p

Prevendo os feriados do estado de São Paulo.

In [1997]:
sp_feriados = holidays.Brazil(state='SP')
def preve_feriado(data):
    data_formatada = data.strftime('%Y-%m-%d')    
    return data_formatada in sp_feriados

In [1998]:

df_p['dias_uteis'] = df_p['ds'].dt.dayofweek < 5 
df_p['feriados'] = df_p['ds'].map(preve_feriado)

Realizando a divisão dos dados em treino e teste, utilizando o SKLearn:

In [1999]:

treino, teste = train_test_split(df_p, test_size=0.0059, shuffle=False)



In [ ]:
treino

In [ ]:
teste

Treinando o modelo comos dados até o dia 26/01/2024, com sazonalidade diária:

In [ ]:
modelo_prop = Prophet(daily_seasonality=True)
modelo_prop.add_regressor("dias_uteis")
modelo_prop.add_regressor("feriados")
modelo_prop.fit(treino)

Ajustando os períodos a serem previstos conforme o tamanho do dataframe de testes, e configurando a coluna booleana de feriados:

In [ ]:
future_prop = modelo_prop.make_future_dataframe(periods=len(teste))
future_prop

In [ ]:
future_prop['dias_uteis'] = df_p['dias_uteis']
future_prop['feriados'] = df_p['feriados']
future_prop

Realizando a previsão:

In [ ]:
forecast_prop = modelo_prop.predict(future_prop)
forecast_prop

In [ ]:
forecast_prop['yhat']

In [ ]:
pd.Series(data=forecast_prop['yhat'].values, index=forecast_prop['ds'])

In [ ]:
predito_prop = forecast_prop.tail(len(teste))[["ds",'yhat']]
predito_prop.rename(columns={'ds':'data_prevista'}, inplace=True)
predito_prop

Criação de um dataframe com os dados de teste e previstos:

In [ ]:
novo_df = pd.DataFrame(teste)
novo_df['previsto'] = predito_prop['yhat'].values
novo_df

Cálculo das métricas de desempenho:

In [ ]:
metrics = calculate_metrics(y_true=teste['y'].values, y_pred=novo_df['previsto'])
print_metrics(metrics)
salvar_metricas('Prophet', metrics)

Esses valores sugerem que o modelo está apresentando um bom desempenho. Uma previsão com MAPE abaixo de 10% já pode ser considerada muito boa em muitos contextos, especialmente em séries temporais voláteis, como a de índices financeiros. O MAE relativamente baixo também sugere que o modelo está realizando previsões com uma margem de erro considerável para aplicações práticas.

In [ ]:
forecast_prop['yhat_upper'].values
forecast_prop['yhat_lower'].values

Criação de dataframe com valor real, previsto (yhat), e intervalos de confiança (yhat_lower e yhat_upper):

In [ ]:
resultado_final  = { 
                    'valor_real': df_p['y'].values, 
                    'valor_previsto': forecast_prop['yhat'].values, 
                    'valor_previsao_maxima': forecast_prop['yhat_upper'].values, 
                    'valor_previsao_minima': forecast_prop['yhat_lower'].values
                    }

df_resultado  = pd.DataFrame(data=resultado_final, index=df_p['ds'])
df_resultado

**Gráficos:** 

In [ ]:
plotarSerie(df_resultado['valor_real'])
plotarSerie(df_resultado['valor_previsto'])
plotarSerie(df_resultado['valor_previsao_maxima'])
plotarSerie(df_resultado['valor_previsao_minima'],titulo="Previsão usando Prophet", legendas=['Valor Real', 'Valor Previsto', 'Intervalo de confiança acima', 'Intervalo de confiança abaixo'])

In [ ]:
plotarSerie(df_resultado['valor_real'].tail(365))
plotarSerie(df_resultado['valor_previsto'].tail(365))
plotarSerie(df_resultado['valor_previsao_maxima'].tail(365))
plotarSerie(df_resultado['valor_previsao_minima'].tail(365),titulo="Previsão dos últimos 365 períodos usando Prophet", legendas=['Valor Real', 'Valor Previsto', 'Intervalo de confiança acima', 'Intervalo de confiança abaixo'])

In [ ]:
modelo_prop.plot(forecast_prop)
plt.legend(loc='best', labels=['Pontos observados','Previsão', 'Intervalo de confiança'])
plt.ylabel('Valor ação')
plt.xlabel('Data')
plt.show()

In [ ]:
resultado_modelos = pd.DataFrame(metricas).T
resultado_modelos.head().sort_values(by="mape", ascending=True)

Até o momento, o modelo com melhor desempenho foi o **XGBRegressor**.

# Aplicação do modelo XGBRegressor com maior tempo

O **XGBRegressor** é um algoritmo de *machine learning* poderoso e eficiente, especialmente utilizado para **problemas de regressão**. Ele funciona combinando múltiplas árvores de decisão (ensemble) de forma sequencial, aprendendo dos erros dos modelos anteriores. Essa abordagem, chamada de *gradient boosting*, permite que o XGBRegressor construa modelos altamente precisos e robustos. 

Embora o XGBRegressor não tenha sido especificamente projetado para séries temporais, ele pode ser adaptado para esse tipo de problema com excelentes resultados. Isso se deve a sua capacidade de:

**Capturar padrões complexos:** As árvores de decisão do XGBoost são capazes de capturar padrões não-lineares e interações entre diferentes características, o que é comum em séries temporais.

**Lidar com grandes volumes de dados:** O algoritmo é otimizado para lidar com grandes conjuntos de dados, o que é frequentemente o caso em séries temporais.

**Alta precisão:** Em diversas competições de machine learning, o XGBoost tem demonstrado ser um dos algoritmos mais precisos.



#### Preparação dos dados

- No método abaixo estamos criando uma função para preparar os dados para o nosso modelo, separamos a data por ano, mes,dia e dia da semana, e renomeamos duas colunas, Último para valor_fechamento e Abertura para valor_abertura

In [2019]:
def create_feature(dado):
    data_frame = pd.DataFrame(dado)
    data_frame['valor_fechamento']  = data_frame['Último']
    data_frame['valor_abertura']  = data_frame['Abertura']
    data_frame['Data'] = pd.to_datetime(data_frame['Data'], format='%d.%m.%Y')
    data_frame['year'] = data_frame["Data"].dt.year
    data_frame['month'] = data_frame["Data"].dt.month
    data_frame['day'] = data_frame["Data"].dt.day
    data_frame['dayofweek'] = data_frame["Data"].dt.dayofweek
    return data_frame

Com a função criada, criamos duas novas variaveis, que é o feature_names no qual será as variaveis que o modelo irá usar do dataframe e o target, que é o valor que o modelo tentará prever.

In [2020]:
feature_names = ["year", "month", "day", "dayofweek","valor_abertura"]
target = "valor_fechamento"

- Em seguida, criaremos um DataFrame de features e o ordenaremos por data para preparar os dados para a divisão em conjuntos de treino e teste.

In [ ]:
df_feature = create_feature(df)
df_feature = df_feature.sort_values(by='Data')
df_feature

- Com o DataFrame de features pronto, podemos dividir os dados em conjuntos de treino e teste. Utilizaremos a função **train_test_split** do scikit-learn para essa tarefa. A variável **X** conterá as features (características) que o modelo utilizará para fazer as previsões, enquanto **y** conterá o target (valor que queremos prever). Definiremos **test_size=0.0059** para que o conjunto de teste represente os últimos 30 dias dos nossos dados (considerando um DataFrame com 5000 registros). Ao configurar **shuffle=False**, garantimos que a ordem temporal dos dados seja preservada, evitando que os dados de teste 'contaminem' o conjunto de treino.

In [ ]:
X = df_feature[feature_names]
y = df_feature['valor_fechamento']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.0450, shuffle=False)
len(X_train), len(X_test)

Separamos treino e teste e temos 4970 registros para teste e 30 para treino

In [ ]:
X_train.head()

Observando as 5 primeiros linhas do treino deu para validar que os dados começaram de 2004 conforme o  esperavamos

In [ ]:
X_train.tail()

Observando suas 5 últimas linhas do treino foi possível notar que os ultimos dias foram janeiro de 2024 conforme o esperado

In [ ]:
X_test

o nosso X_test tambem ficou adequado, começando final de janeiro até dia 12 de março

Criamos um modelo XGBRegressor configurado para minimizar o erro quadrático médio ("reg:squarederror").
- Treinamos o modelo com os dados de treino (`X_train`, `y_train`). O modelo aprende a relação entre as features (`X_train`) e o target (`y_train`).
- Utilizamos o modelo treinado para fazer previsões nos dados de teste (`X_test`). A variável `predict_values` armazena as previsões do modelo para cada ponto no conjunto de teste.
- `predict_values` contém as previsões do modelo para o target nos dados de teste, que podemos comparar com o target real para avaliar a performance do modelo.


In [2026]:
reg = xgb.XGBRegressor(objective="reg:squarederror")
reg.fit(X_train, y_train)

predict_values = reg.predict(X_test)

Com os dados já previstos, iremos calcular as metricas para entender como foi a performance do modelo

In [ ]:
metrics = calculate_metrics(y_true=predict_values, y_pred=y_test.values)
print_metrics(metrics)
salvar_metricas('XGBRegressor', metrics)

O erro médio absoluto (MAE) indica que, em média, as previsões estão desviando cerca de 1.68 unidades do valor real. 

O erro quadrático médio (MSE) sugere que os erros maiores estão sendo penalizados.

O erro percentual absoluto médio (MAPE) indica um erro médio de aproximadamente 1.32%.

Agora iremos reconstruir a data da serie treino para que seja possível visualizar os dados com os valores previstos

In [ ]:
X_train["data"] = X_train["year"].map(str) + "-" + X_train["month"].map(str) + "-" + X_train["day"].map(str)
serie_treino = pd.Series(data=y_train.values, index=pd.to_datetime(X_train['data']))
serie_treino

Construindo as datas dos valores previsto

In [ ]:
X_test["data"] = X_test["year"].map(str) + "-" + X_test["month"].map(str) + "-" + X_test["day"].map(str)
serie_prevista_xgb = pd.Series(data=predict_values, index=pd.to_datetime(X_test['data']))
serie_prevista_xgb


In [ ]:
plotarSerie(serie_treino)
plotarSerie(serie_prevista_xgb,titulo="Previsão bolsa de valores usando XGB Regressor", legendas=['Valor Real', 'Valor Previsto'])


In [ ]:
df_ibovespa

In [ ]:
plotarSerie(serie_treino[-500:])
plotarSerie(serie_prevista_xgb, titulo="Previsão bolsa de valores usando XGB Regressor", legendas=['Valor Real', 'Valor Previsto'])
